In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt

#RegEx
import re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1608046971644_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Query some reviews
query_1 = "select distinct r.ugc_id,  r.review_headline, r.review_comments, r.reviewer_nickname, r.reviewer_location, \
case when (contains(r.observations, 'PR') )  then 1 else 0 end as PR \
from analytics.review as r \
where r.locale in ('en_US','en_GB','en_CA') and contains(r.observations, 'PM') \
and r.created_date > '2020-01-01' \
limit 1000"


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
reviews = pd.read_sql(query_1, conn)
reviews.head(20)
#reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID  ... PR
0   314075579  ...  0
1   314075467  ...  0
2   314091178  ...  0
3   314157366  ...  0
4   314225919  ...  0
5   314222457  ...  0
6   314232053  ...  0
7   314216224  ...  0
8   319248172  ...  0
9   317725200  ...  0
10  314351901  ...  0
11  318543473  ...  0
12  314075256  ...  0
13  314233933  ...  0
14  314352263  ...  0
15  314352451  ...  0
16  314359766  ...  0
17  314364805  ...  0
18  314368502  ...  0
19  314368840  ...  0

[20 rows x 6 columns]

In [43]:

reviews['ALL_TEXT'] =  reviews['REVIEW_HEADLINE'] + ' '+reviews['REVIEW_COMMENTS']
#Use set_option so text doesn't truncate

#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
reviews.head()

#reviews.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID  ...                                           ALL_TEXT
0  267155945  ...  Excellent for the price. Great for me.  Assy. ...
1  267110279  ...  Great starter rower I am slowly trying to buil...
2  267180402  ...  Radiant Heater Reflector Considerations We hav...
3  267180426  ...  Surprisingly good laptop value I had been look...
4  267066742  ...  I recommend Recommended by my dentist, does a ...

[5 rows x 7 columns]

In [44]:
reviews['regex_price'] = reviews['ALL_TEXT'].str.contains('\d{4}')== True


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/local/lib64/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/usr/local/lib64/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)

In [48]:
#reviews['regex_PII'] = 0
#reviews.loc[(reviews['regex_creditcard']==True) | (reviews['regex_socsec']==True) | (reviews['regex_phone10']==True) \
#        | (reviews['regex_USaddress_comments']==True) | (reviews['regex_USaddress_headline']==True), 'regex_PII'] = 1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID  ... regex_PII
0  267155945  ...         0
1  267110279  ...         0
2  267180402  ...         0
3  267180426  ...         0
4  267066742  ...         0

[5 rows x 16 columns]

In [50]:
FP = reviews[(reviews['PII'] == 0) & (reviews['regex_PII'] == 1)]
FN = reviews[(reviews['PII'] == 1) & (reviews['regex_PII'] == 0)]
TP = reviews[(reviews['PII'] == 1) & (reviews['regex_PII'] == 1)]
TN = reviews[(reviews['PII'] == 0) & (reviews['regex_PII'] == 0)]
FP['UGC_ID'].count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

19739

In [51]:
FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

125304

In [52]:
TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8745

In [53]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

17715798

In [57]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

test = FP[['ALL_TEXT','PII','regex_PII']]
test.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("REVIEWER_NICKNAME", StringType(), True)\
,StructField("REVIEWER_LOCATION", StringType(), True)\
,StructField("PII", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("regex_creditcard", BooleanType(), True)\
,StructField("regex_soc_sec", BooleanType(), True)\
,StructField("regex_phone10", BooleanType(), True)\
,StructField("regex_email", BooleanType(), True)\
,StructField("regex_handle", BooleanType(), True)\
,StructField("regex_USaddress_loc", BooleanType(), True)\
,StructField("regex_USaddress_comments", BooleanType(), True)\
,StructField("regex_USaddress_headline", BooleanType(), True)\
,StructField("regex_PII", IntegerType(), True)])   



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
reviews = spark.createDataFrame(reviews, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : 'APP_SPARK_PROD',
  'sfPassword' : '>6txhcQ*7&^Qrq`j',
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
reviews.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","test1").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o206.save.
: net.snowflake.client.jdbc.SnowflakeSQLException: Cannot perform CREATE STAGE. This session does not have a current database. Call 'USE DATABASE', or use a qualified name.
	at net.snowflake.client.jdbc.SnowflakeUtil.checkErrorAndThrowExceptionSub(SnowflakeUtil.java:152)
	at net.snowflake.client.jdbc.SnowflakeUtil.checkErrorAndThrowException(SnowflakeUtil.java:77)
	at net.snowflake.client.core.StmtUtil.pollForOutput(StmtUtil.java:495)
	at net.snowflake.client.core.StmtUtil.execute(StmtUtil.java:372)
	at net.snowflake.client.core.SFStatement.executeHelper(SFStatement.java:575)
	at net.snowflake.client.core.SFStatement.executeQueryInternal(SFStatement.java:265)
	at net.snowflake.client.core.SFStatement.executeQuery(SFStatement.java:203)
	at net.snowflake.client.core.SFStatement.execute(SFStatement.java:874)
	at net.snowflake.client.jdbc.SnowflakeStatementV1.executeQueryInternal(SnowflakeStatementV1.java:259)
	at net.snowflake.client.jdbc.Snowfla

In [8]:
#Ok if it doesn't get phone # with 7 digits bc no one could call without area code anyway
#Would mark any location with Name, only want addresses with street info in them
text = "My name is Apple Smith can be contacted on example@example.com, or 1800 555-5555 or 123-3421 or +1(800)452-1234 \
soc sec num 123-02-3240 or without dashes 123023240 or with periods 123.02.3240 \
or at 414 Wrightwood Ave Chicago, IL or in Chicago or in Chicago, IL. \
I work at The North Face. Is this a Name? \
Credit card number is 1231 1231 1231 1231 or 1231-1231-1231-1231. \
Website is http://www.amazon.com or www.amazon.com or amazon.com"
 

# Replaces the phone number and email addresse with anonymous IDs.
scrubadub.clean(text)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'My name is {{NAME}} {{NAME}} can be contacted on {{EMAIL}}, or {{PHONE}} or 123-3421 or {{PHONE}} soc sec num {{SSN}} or without dashes 123023240 or with periods {{SSN}} or at 414 {{NAME}} {{NAME}} {{NAME}}, IL or in {{NAME}} or in {{NAME}}, IL. I work at The {{NAME}} {{NAME}}. Is this a Name? {{NAME}} card number is 1231 1231 1231 1231 or 1231-1231-1231-1231. {{NAME}} is {{URL}} or {{URL}} or amazon.com'

In [9]:
scrubber = scrubadub.Scrubber()
scrubber.remove_detector('name')
scrubber.clean(text)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'My name is Apple Smith can be contacted on {{EMAIL}}, or {{PHONE}} or 123-3421 or {{PHONE}} soc sec num {{SSN}} or without dashes 123023240 or with periods {{SSN}} or at 414 Wrightwood Ave Chicago, IL or in Chicago or in Chicago, IL. I work at The North Face. Is this a Name? Credit card number is 1231 1231 1231 1231 or 1231-1231-1231-1231. Website is {{URL}} or {{URL}} or amazon.com'

In [14]:
txt = "My credit card is 1234 123412341234"
x = re.search('\d{4}(-|\.|\s)?\d{4}(-|\.|\s)?\d{4}(-|\.|\s)?\d{4}', txt)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
x

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<re.Match object; span=(18, 35), match='1234 123412341234'>